#### LangChain Essentials Course

# LangChains Expression Language

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Expression Langauge (LCEL), abstracting a full chain and understanding how it will work. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

## Choosing your Model

This example is split into two versions - The [Ollama version](), allowing us to run our LLM locally without needing any external services or API keys. The [OpenAI version](https://github.com/aurelio-labs/agents-course/blob/main/04-langchain-ecosystem/01-langchain-essentials/01-langchain-intro-openai.ipynb) uses the OpenAI API and requires an OpenAI API key.

## Initializing OpenAI's gpt-4o-mini

We start by initializing the OpenAI model, fine-tuned for instruction following. We `pull` the model from OpenAI by switching to our terminal and executing:

```
pip install openai
```

Once the model libary has finished downloading, we initialize it in LangChain using the `ChatOpenAI` class:

In [129]:
# OpenAI key to use the model
OPENAI_API_KEY = ""

openai_model = "gpt-4o-mini"

import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [130]:
from langchain.chat_models import ChatOpenAI

# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model=openai_model, openai_api_key = OPENAI_API_KEY)

# For unique creative responses
creative_llm = ChatOpenAI(temperature=0.9, model=openai_model, openai_api_key = OPENAI_API_KEY)

## Traditional Chains vs LCEL Chains

In this section we're going to dive into a basic example comparing the main difference between the two chains. For this we will use a basic example of finding the user's report, where the user must input a specific topic, and then the LLM will look and return a report on the specified topic.

In [131]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [132]:
from langchain import PromptTemplate

prompt_template = "Give me a small report of {topic}"

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)

Here is a standard LLMChain, this has the basic function like properties where we would call a name of the function, and pass in parameters to adjust the function, in this case, prompt, llm and output_parser, where the prompt will be used by the model, and then the result of the model will be used by the output parser.

In [134]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt = prompt,
    llm = llm,
    output_parser = output_parser
)

In [135]:
result = chain.invoke("AI")
print(result)

{'topic': 'AI', 'text': '### Report on Artificial Intelligence (AI)\n\n#### Introduction\nArtificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies and methodologies, including machine learning, natural language processing, robotics, and computer vision. AI has become a transformative force across multiple sectors, influencing how businesses operate, how services are delivered, and how individuals interact with technology.\n\n#### Current Trends in AI\n1. **Machine Learning and Deep Learning**: These subfields of AI focus on algorithms that allow computers to learn from and make predictions based on data. Deep learning, a subset of machine learning, utilizes neural networks with many layers to analyze various forms of data.\n\n2. **Natural Language Processing (NLP)**: NLP enables machines to understand, interpret, and respond to human language. Applications include chatbots

This is a LCEL chain, as you can see, this initially appears to be abit 'hacky' but the abstraction allows for us to skip calling a function and pass our variables into eachother instead.

In [136]:
lcel_chain = prompt | llm | output_parser

In [137]:
result = lcel_chain.invoke("AI")
print(result)

### Report on Artificial Intelligence (AI)

#### Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies and methodologies, including machine learning, natural language processing, robotics, and computer vision. AI has become a transformative force across multiple sectors, influencing how businesses operate, how services are delivered, and how individuals interact with technology.

#### Current Trends in AI
1. **Machine Learning and Deep Learning**: These subfields of AI focus on algorithms that allow computers to learn from and make predictions based on data. Deep learning, a subset of machine learning, utilizes neural networks with many layers to analyze various forms of data.

2. **Natural Language Processing (NLP)**: NLP enables machines to understand, interpret, and respond to human language. Applications include chatbots, virtual assistants, and languag

## How Does LCEL Work?

The concept is reasonably simple, you start on the far left side of the line, look at the first variable, and the output to that variable is passed into the next variable, before we had ***prompt | llm | output_parser***, we can see that the prompt, feeds into the model, then the model result feeds into the output parser.

When we use the pipe operator **|** we are basically looking for a or function, this is where we can find a chained functionallity to the variable.

Let's make a basic runnable class to show you the basics of how this works.

In [138]:
class Runnable:
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

We firstly want to make a bunch of random functions, in this case we will do some simple maths with each function.

In [139]:
def add_five(x):
    return x+5
def sub_five(x):
    return x-5
def mul_five(x):
    return x*5

Now we want to coat our functions with the runnable so that the or function can be identified.

In [140]:
add_five = Runnable(add_five)
sub_five = Runnable(sub_five)
mul_five = Runnable(mul_five)

Now as you can see we can chain together the 3 functions we made just earlier using the or function, and if we switch the or functions out for the pipe operator, it does exactly the same.

In [141]:
chain = (add_five).__or__(sub_five).__or__(mul_five)
chain(3)

15

In [142]:
chain = add_five | sub_five | mul_five
chain(3)

15

## LCEL Parallel Use-Case Scenario

In this section we will go over how we can use LCEL's parallel capabilities.

To start us of with, we will have two statements, one side telling the AI the month and day Josh was born, and the other telling the AI the year Josh was born. We will then embed the statements and feed them into AI together.

In [143]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embedding = OpenAIEmbeddings()
vecstore_a = DocArrayInMemorySearch.from_texts(
    ["half the info is here", "Joshs' birthday is June the 12th"], 
    embedding=embedding
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    ["the other half of the info is here", "Josh was born in 2002"], 
    embedding=embedding
)

Here you can see the prompt does have three inputs, two for context and one for the question itself.

In [144]:
prompt_str = """ using the following context answer the question
Context: 
{context_a}
{context_b}

Question:
{question}

Answer: """

In [145]:
from langchain.prompts.chat import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(prompt_str)

Here we are wrapping our vector stores as retrievers so they can be fitted into one big retrieval variable to be used by the prompt.

In [146]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b, "question": RunnablePassthrough()
    }
)

In [147]:
chain = retrieval | prompt | llm | output_parser

In [148]:
result = chain.invoke("What was the date when Josh was born")

In [149]:
result

'Josh was born on June 12, 2002.'

## LangChain's RunnableLambdas

You can use arbitrary functions as Runnables. This is useful for formatting or when you need functionality not provided by other LangChain components, and custom functions used as Runnables are called RunnableLambdas.

Note that all inputs to these functions need to be a SINGLE argument. If you have a function that accepts multiple arguments, you should write a wrapper that accepts a single dict input and unpacks it into multiple arguments.

In [150]:
from langchain_core.runnables import RunnableLambda

Here we can make some custom functions that do simple maths again, and see that RunnableLambdas can compile and output the correct results.

In [151]:
def add_five(x):
    return x+5
def sub_five(x):
    return x-5
def mul_five(x):
    return x*5

In [152]:
add_five = RunnableLambda(add_five)
sub_five = RunnableLambda(sub_five)
mul_five = RunnableLambda(mul_five)

In [153]:
chain = add_five | sub_five | mul_five

In [154]:
chain.invoke(3)

15

Now we want to try something a little more testing, so this time we will generate a report, and we will try and edit that report using this functionallity.

In [155]:
prompt_str = "give me a small report about {topic}"
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_str
)

In [156]:
chain = prompt | llm | output_parser

In [157]:
result = chain.invoke("AI")

In [158]:
print(result)

### Report on Artificial Intelligence (AI)

#### Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies and methodologies, including machine learning, natural language processing, robotics, and computer vision. AI has rapidly evolved over the past few decades, becoming an integral part of various industries and daily life.

#### Historical Background
The concept of AI dates back to the mid-20th century, with pioneers like Alan Turing and John McCarthy laying the groundwork. The term "artificial intelligence" was coined in 1956 during the Dartmouth Conference, which is considered the birth of AI as a field. Initial progress was slow, leading to periods of reduced funding and interest known as "AI winters." However, advancements in computing power, data availability, and algorithmic techniques have led to a resurgence in AI research and applications since the 

Here we are making two functions, one that will get rid of the introduction to the AI finding the information, instead we will just see the information, and word replacer that will replace AI with Josh!

In [159]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x
    
old_word = "AI"
new_word = "Josh"

def replace_word(x):
    return x.replace(old_word, new_word)

Lets wrap these functions and see what the output is!

In [161]:
extract_fact = RunnableLambda(extract_fact)
replace_word = RunnableLambda(replace_word)

In [162]:
chain = prompt | llm | output_parser | extract_fact | replace_word

In [163]:
result = chain.invoke("AI")

In [164]:
print(result)

#### Introduction
Artificial Intelligence (Josh) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies and methodologies, including machine learning, natural language processing, robotics, and computer vision. Josh has rapidly evolved over the past few decades, becoming an integral part of various industries and daily life.
#### Historical Context
The concept of Josh dates back to the mid-20th century, with pioneers like Alan Turing and John McCarthy laying the groundwork. The term "artificial intelligence" was coined in 1956 during the Dartmouth Conference, which is considered the birth of Josh as a field. Early Josh research focused on problem-solving and symbolic methods, but progress was slow due to limited computational power and data.
#### Current Trends
1. **Machine Learning (ML)**: A subset of Josh that enables systems to learn from data and improve over time without explicit programming. D